In [3]:
import torchaudio
import noisereduce as nr
# from torchaudio.functional import *
import torch as tr
import matplotlib.pyplot as plt

C:\Users\Joseph\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
x1, sr = torchaudio.load('../normalized/sensor_1.wav')
x2, sr = torchaudio.load('../normalized/sensor_2.wav')
x3, sr = torchaudio.load('../normalized/sensor_3.wav')
x_test, sr = torchaudio.load('../normalized/test_rec.wav')
x_clean, sr = torchaudio.load('../uniform_mix_reduced/uni_mix_reduced_output.wav')
print(f"Sample length is {x_test.shape[1] / sr:2.1f}s")

Sample length is 217.6s


In [69]:
def windows(length, window_size):
    for j in range(0, length, window_size):
        yield j, j + window_size - 1

In [63]:
def windowed_energy_loss(xa, xb, N, window_length=1):
    Ea = tr.sqrt(tr.sum(xa**2))
    Eb = tr.sqrt(tr.sum(xb**2))
    y = []
    for i, j in windows(N, int(window_length * sr)):
        a = xa[i:j] / Ea # Normalize inputs to make independent of volume
        b = xb[i:j] / Eb
        y.append(tr.sqrt(tr.sum((a-b)**2))) # root-mean-square-error
    y = tr.Tensor(y)
    return tr.sum(y)

In [64]:
kargs = {'N':x_test.shape[1], 'window_length':.5}

In [65]:
print(windowed_energy_loss(x1[0], x_test[0], **kargs))

tensor(25.9527)


In [66]:
print(windowed_energy_loss(x2[0], x_test[0], **kargs))

tensor(26.5878)


In [67]:
print(windowed_energy_loss(x3[0], x_test[0], **kargs))

tensor(26.7217)


In [68]:
print(windowed_energy_loss(x_test[0], x_clean[0], **kargs))

tensor(23.6870)
